<a href="https://colab.research.google.com/github/cisimon7/Non-Contact-Measurement-of-Cable-Profile/blob/main/utilities/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch

In [ ]:
def catenary(x, a):
    return a * torch.cosh((x + 1) / a)          # Assume the form of catenary

"""
Is it assumed that alpha = 1 and beta = 0 ?
if true, why is that?

I guess this is the real life form of the cable, which can actually be any shape 
but must be of the catenary form.

Our goal is to be able to find this equation, i.e a = a, alpha = 1 and beta = 0
"""

In [ ]:
def to_image(vector_x, camera_matrix):
    """
    Projects 3D vectors to 2D image coordinates given camera configuration
    This function imitate how points are formed from 3D world to a camera 2D plane

    camera_matrix: a 3X3 matrix defined by the camera intrinsic parameters
    The extrinsic parameters have been fixed here
    """
    R = torch.Tensor(np.identity(3))            # Camera Rotation matrix, assume R is identity (3 X 3) 
    t = torch.Tensor([0, 0, 100]).unsqueeze(1)  # 3D coordinate; assume to be [0 0 100]. This means camera is facing object directly with z representing distance of camera to object 

    ext_params = torch.cat((R, t), 1)           # Get Q|t. (4 X 3)
    img = camera_matrix @ ext_params @ vector_x # C[Q|T]x. Returns w[x, y, 1] where w is the scale factor. Possible Error: (vector_z, 1) @ ext_params @ camera_matrix
    img = img / img[-1]                         # divide by z to get coordinates 
    return img

# https://www.mathworks.com/help/vision/ug/camera-calibration.html

In [ ]:
"""
Why adding noise ???? 
Because in real life, our measurement cannot be very accurate
"""
def add_noise(*vector_u):
    # Adds noise from N(0, 1) to all vectors in the input
    return list(map(lambda x: x + torch.normal(0, 1, (2,)), vector_u))

In [ ]:
def calculate_error(u, u_hat, error_func=lambda x: torch.norm(x, 2)):
  """
  Calculates error function, given by the user (second norm by default)
  u     : actual image coordinate in 2D world 
  u_hat : noisy image coordinate in 2D world 
  """
  return error_func(u - u_hat)

In [ ]:
def calculate_step(a, x_1, x_2, camera_matrix):
    # One step of the optimization

    # Base on guess, this is representing the y position of the real life cable
    y_1 = catenary(x_1, a)           
    y_2 = catenary(x_2, a)

    # Stack values of xs and ys to get the 3D vector (z=0 as they are in the same plane)
    zero, one = torch.tensor(0, dtype=torch.float32), torch.tensor(1, dtype=torch.float32)

    # Notice that the z value has been set to zero, what does this mean to how we place our axis during the actual measurement
    # Why one is added can be seen in the camera calibration link
    vector_x1 = torch.stack((x_1, y_1, zero, one))
    vector_x2 = torch.stack((x_2, y_2, zero, one))

    # Project to the image space
    u1 = to_image(vector_x1, camera_matrix)[:2]
    u2 = to_image(vector_x2, camera_matrix)[:2]

    return u1, u2

In [ ]:
# Class for simulation steps
class Simulation:
    # We need initial assumption of a, x1,x2 and camera matrix
    def __init__(self, a, x1, x2, camera_matrix):
        self.camera_matrix = camera_matrix

        # Calculate the target 'tilde u1' and 'tilde u2' ???
        self.perfect_u1, self.perfect_u2 = add_noise(*calculate_step(a, x1, x2, camera_matrix))

        # Trainables a, x1 and x2
        self.a = torch.tensor(1, dtype=torch.float32, requires_grad=True)
        self.x_1 = torch.tensor(0, dtype=torch.float32, requires_grad=True)
        self.x_2 = torch.tensor(0, dtype=torch.float32, requires_grad=True)

        # Init optimizer and learning rate scheduler
        self.optimizer = torch.optim.SGD([self.a, self.x_1, self.x_2], lr=1e-2)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min', factor=0.99)

    def calculate_step(self):
        # One step of the optimization
        y_1 = catenary(self.x_1, self.a)  # Calculate y1 and y2 values using assumed form of the catenary
        y_2 = catenary(self.x_2, self.a)

        # Stack values of xs and ys to get the 3D vector (z=0 as they are in the same plane)
        zero, one = torch.tensor(0, dtype=torch.float32), torch.tensor(1, dtype=torch.float32)
        vector_x1 = torch.stack((self.x_1, y_1, zero, one))
        vector_x2 = torch.stack((self.x_2, y_2, zero, one))

        # Project to the image space
        u1 = to_image(vector_x1, self.camera_matrix)[:2]
        u2 = to_image(vector_x2, self.camera_matrix)[:2]

        return u1, u2

    def simulate(self, iters=10000, verbosity=10000):
        # Simulation of convergence
        for i in range(iters):
            self.optimizer.zero_grad()

            # Get current u's
            new_u1, new_u2 = calculate_step(self.a, self.x_1, self.x_2, self.camera_matrix)

            # Calculate loss
            loss = (calculate_error(self.perfect_u1, new_u1) + calculate_error(self.perfect_u2, new_u2)) / 2
            if i % verbosity == 0:
                print(f'Epoch {i}. Loss {loss}')

            loss.backward()
            self.optimizer.step()
            self.scheduler.step(loss)
            
        return self.a, self.x_1, self.x_2, new_u1, new_u2